# This is the final project in the frames of the "Applied Data Science Capstone"

The concept of this project is to compare the neighborhoods of New York and Toronto cities. Some analysis has already been done in Week 3, so it won't be so difficult.
Similarities and differences between the neighborhoods of a city or the two cities can be found in terms of shops and others.

#### Exploring the data

The data of the project will be taken by the cloud of IBM for New York city and from Wikipedia for Toronto. The process will be the same as in Week 3.